In [1]:
!ls drive

ls: cannot access 'drive': No such file or directory


In [1]:
!apt install jq
!pip install git+https://github.com/gmagannaDevelop/segnet.git@mru_tests

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 7 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [111 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 jq amd64 1.5+dfsg-2 [45.6 kB]
Fetched 276 kB in 1s (379 kB/s)
Selecting previously unselected package libonig4:amd64.
(Reading database ... 145674 files and directories currently installed.)
Preparing to unpack .../libon

In [2]:

import os

###############################################################
import tensorflow as tf

import segnet.metrics as mts
 
from segnet.models import unet
from segnet.models import multiresunet as mru
from segnet.models import multiresunet2 as mru2
from segnet.models import multiresunet3 as mru3
from segnet.utils.Segmed import Segmed

### Data-related
from google.colab import drive, files
drive.mount('/content/drive/')
###############################################################
### Not an import, but mandatory to be defined here :
root_dir = "drive/My Drive/Gus_Servicio_Profesional"
log_dir  = os.path.join(root_dir, "Logs")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
dataset_paths = { 
  "isbi":          "drive/My Drive/Gus_Servicio_Profesional/Colab_data/ISBI_neural/structured", 
  "colonoscopy":   "drive/My Drive/Gus_Servicio_Profesional/Colab_data/colonoscopy",   # Full original
  "dermoscopy80":  "drive/My Drive/Gus_Servicio_Profesional/Colab_data/dermoscopy80",  # reduced to 80 images
  "dermoscopy150": "drive/My Drive/Gus_Servicio_Profesional/Colab_data/dermoscopy150", # reduced to 150, 
  "chinese1":      "drive/My Drive/Gus_Servicio_Profesional/Colab_data/Dataset 2"      # Chinese dataset
} 

In [0]:
optimizers = {
    "chinese": tf.keras.optimizers.SGD(learning_rate=0.06, momentum=0.2, nesterov=False),
    "Original Adam": tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999, epsilon=10e-8)
}

my_compiling_kw = {
    'optimizer': optimizers["Original Adam"],
    'loss': 'binary_crossentropy',
    'metrics': [
      mts.jaccard_index, mts.dice_coef,
      mts.O_Rate, mts.U_Rate, mts.Err_rate
    ]
}


In [0]:
dataset = dataset_paths["dermoscopy80"]

In [6]:
len(os.listdir(os.path.join(dataset, "msks/masks")))

81

In [0]:
my_hyper_params = {
    'batch_size': 20,
    'epochs': 15,
    'steps_per_epoch': 4
}

In [8]:
architectures = {
  "Unet": unet(),
  "MultiResUNet Edwin": mru.MultiResUnet(),
  "MultiResUNet Gustavo": mru2.MultiResUNet(),
  "MultiResUNet Original": mru3.MultiResUnet()
}

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
models = {
    key: Segmed(
        model = architectures[key],
        name = key,
        base_dir = log_dir,
        data_path = dataset_paths["dermoscopy80"],
        author = "Gustavo Magaña"
    )
    for key in architectures.keys()
}

In [11]:
models

{'MultiResUNet Edwin': <segnet.utils.Segmed.Segmed at 0x7f4d0c9af5f8>,
 'MultiResUNet Gustavo': <segnet.utils.Segmed.Segmed at 0x7f4d0c8458d0>,
 'MultiResUNet Original': <segnet.utils.Segmed.Segmed at 0x7f4cb5d9b278>,
 'Unet': <segnet.utils.Segmed.Segmed at 0x7f4cb5d9b080>}

In [0]:
for model in models.values():
  model.comment(" Retrying using GPU accelerator, few epochs and the smallest dataset that I have. ")

In [13]:
for model in models.values():
  model.train(
    compiling_kw = my_compiling_kw,
    hyper_params = my_hyper_params
  )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 64 images belonging to 1 classes.
Found 64 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Found 16 images belonging to 1 classes.
Epoch 1/15
4/4 [==============================] - 14s 4s/step - loss: 0.5458 - jaccard_index: 0.1870 - dice_coef: 0.3150 - O_Rate: 146970000.0000 - U_Rate: 0.0000e+00 - Err_rate: 146970000.0000

Epoch 00001: val_jaccard_index improved from -inf to 0.18697, saving model to drive/My Drive/Gus_Servicio_Profesional/Logs/Unet-(Gustavo Magaña)-(2020-02-01 19:52:46)/Unet-(Gustavo Magaña)-(2020-02-01 19:52:46)-model.h5
4/4 [==============================] - 55s 14s/step - loss: 3.3493 - jaccard_index: 0.1718 - dice_coef: 0.2793 - O_Rate: 206482256.0000 - U_Rate: 0.0000e+00 - Err_rate: 206482256.0000 - val_loss: 0.5458 - val_jaccard_index: 0.1870 - val_dice_coef: 0.3150 - val_O_Rate: 146970000.0000 - val_U_Rate: 0.0000e+00 - val_Err_rate: 146970

ResourceExhaustedError: ignored